## Project Purpose ##
The purpose of this project is to predict sunspots for a given date using RNN-based model.

In [16]:
# Import necessary libraries 
import numpy as np # for numerical operations
import pandas as pd # for dataset processing 
from keras.models import Sequential # building neural networks with simple structure
from keras.layers import SimpleRNN, Dense, Dropout, Activation # recurrsive neural network
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError # performancemetrics of a model
# This allows to process the time-series dataset.
from sklearn.preprocessing import MinMaxScaler

In [18]:
# Load the dataset
df = pd.read_csv("monthly-sunspots.csv", delimiter=",")
df.set_index(df.columns[0], inplace = True)
df

,Sunspots
Month,
1749-01,58.0
1749-02,62.6
1749-03,70.0
1749-04,55.7
1749-05,85.0
...,...
1983-08,71.8
1983-09,50.3
1983-10,55.8


In [20]:
# Preprocessing 
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# number of prior observations
time_steps = 12

# Create sequences for training
X, y = [], []
for i in range(len(df_scaled) - time_steps):
    X.append(df_scaled[i : i + time_steps])
    y.append(df_scaled[i + time_steps])
X, y = np.array(X), np.array(y)

# Split dataset into training and test sets 
train_size = int(len(X) * 0.80)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]



In [22]:
# Function to build model 
def build_model():
    model = Sequential()  # Initialize the model as a sequential model
    layers = [1, 50, 100, 1]  # Define the size of each layer in the model
    # Add layer 
    model.add(SimpleRNN(layers[1], input_shape=(time_steps, layers[0]), return_sequences=True))
    model.add(Dropout(0.2)) 
    # Add another SimpleRNN layer
    model.add(SimpleRNN(layers[2], return_sequences=False))
    model.add(Dropout(0.2))
    # Output layer with linear activation function
    model.add(Dense(layers[3])) 
    model.add(Activation("linear"))  
    # Compile the model with mean squared error loss and RMSprop optimizer
    model.compile(loss="mse", optimizer="adam", metrics=['mae'])
    return model

In [24]:
epochs = 15
# Calling the model building function to construct 
model = build_model()

# Training the model on the preprocessed training data.
model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_split=0.2)

# Use the model to predict the output for the testing dataset.
predicted = model.predict(X_test)

# The predicted values are reshaped to match the size of the testing target data for evaluation purposes.
predicted = np.reshape(predicted, (predicted.size,))

# The trained model is evaluated using the testing dataset to calculate the loss and other metrics like Mean Squared Error (MSE), Mean Absolute Error (MAE)
scores = model.evaluate(X_test, y_test, batch_size=512)


Epoch 1/15
15/15 [==============================] - 1s 11ms/step - loss: 0.0795 - mae: 0.1981 - val_loss: 0.0091 - val_mae: 0.0706
Epoch 2/15
15/15 [==============================] - 0s 5ms/step - loss: 0.0273 - mae: 0.1204 - val_loss: 0.0100 - val_mae: 0.0752
Epoch 3/15
15/15 [==============================] - 0s 5ms/step - loss: 0.0152 - mae: 0.0893 - val_loss: 0.0039 - val_mae: 0.0449
Epoch 4/15
15/15 [==============================] - 0s 5ms/step - loss: 0.0103 - mae: 0.0724 - val_loss: 0.0039 - val_mae: 0.0461
Epoch 5/15
15/15 [==============================] - 0s 5ms/step - loss: 0.0113 - mae: 0.0754 - val_loss: 0.0040 - val_mae: 0.0461
Epoch 6/15
15/15 [==============================] - 0s 5ms/step - loss: 0.0100 - mae: 0.0720 - val_loss: 0.0037 - val_mae: 0.0442
Epoch 7/15
15/15 [==============================] - 0s 5ms/step - loss: 0.0090 - mae: 0.0666 - val_loss: 0.0034 - val_mae: 0.0417
Epoch 8/15
15/15 [==============================] - 0s 6ms/step - loss: 0.0082 - mae: 0.0